In [1]:
import pandas as pd
import requests # библиотека предназначена для работы с запросами к серверу
from bs4 import BeautifulSoup # для раскрывания и работы с HTML файлом
import json
from pprint import pprint
import requests


In [2]:
def fetch_html(url): # функция которая забирает HTML c сайта
    try:
        response = requests.get(url) # берет и получает ответ от сервера
        response.raise_for_status()  # Check if the request was successful печатает статус 
        return response.text  # Return the HTML content of the page возврощает контент HTML
    except requests.exceptions.RequestException as e:  # логика которая выполняется если произойдет ошибка
        print(f"An error occurred: {e}")
        return None

In [3]:
URL = "https://www.sravni.ru/vklady/top/?sortProperty=rate&conditions=haveCapitalization"
response = fetch_html(URL)
html = BeautifulSoup(response, 'html.parser') 
#BeautifulSoup - cервер отчечает всегда HTML файлом, преобразует HTML из строки в свой формат, 
#это позволяет обращаться к элементам гараздо эффективнее используя имена блоков, ссылки, по тегам работать.

In [4]:
banks_data = html.find_all(class_="style_wrapper__c9ahs style_commonMargin__GJPlL") #find_all - возврощаем все блоки с классом
banks_data

[<div class="style_wrapper__c9ahs style_commonMargin__GJPlL"><div class="Card_wrapper__0WEM7"><div class="lazyload-wrapper"><div class="lazyload-placeholder"></div></div><div class="Card_card__kBcKQ Card_advertisingCard__FQP87"><div class="_1l6wwe5" style='--grid-template-areas:"logo licenseGroup firstCellGroup secondCellGroup thirdCellGroup detailsToggle button"
           "logo ratingGroup firstCellGroup secondCellGroup thirdCellGroup detailsToggle button";--grid-template-columns:44px minmax(160px, 2fr) minmax(140px, 180px) minmax(140px, 180px) minmax(140px, 180px) 46px 178px;--grid-template-rows:repeat(2, max-content);gap:0 16px'><div class="_15zg3r5" style="width:44px;height:44px;grid-area:logo"><div class="_qxd09l"><img alt="Транскапиталбанк" class="_1d6omtb" loading="lazy" src="https://s91588.cdn.ngenix.net/organizations/logotypes/square/5bb4f767245bc22a520a60cd.svg" title="Транскапиталбанк"/></div></div><div class="_1l6wwe5" style='--grid-template-areas:"caption license";--grid-

In [5]:
for bank_data_ in banks_data:     # bank_data - переменная в цикле, которая пробигается в списке div
    try:
        bank_name = bank_data_.find(class_='_jp36ng').text
        interest_rate = bank_data_.find_all(class_='_hxz9o6 _18hwa2n')[0].text
        period_from = bank_data_.find_all(class_='_hxz9o6 _18hwa2n')[1].text
        loan_sum = bank_data_.find_all(class_='_hxz9o6 _18hwa2n')[2].text.replace('\xa0', '')
        print('\n', bank_name, '|' , interest_rate, '|', period_from, '|',loan_sum)
    except:
        print('Ошибка парсинга по классу')


 Транскапиталбанк | 22.35 % | 367 дней | 100000 – 3000000 ₽

 Московский Кредитный Банк | до 24 % | 95 – 1100 дней | 10000 – 3000000 ₽

 Альфа-Банк | до 22 % | 92 – 1095 дней | от 50000 ₽

 Т-Банк | до 24.13 % | 2 – 24 месяцев | от 50000 ₽

 МТС Банк | до 22.55 % | 91 – 1096 дней | от 10000 ₽

 Яндекс Банк | до 22 % | 91 – 730 дней | 10000 – 10000000 ₽

 Газпромбанк | до 22.02 % | 181 день | от 15000 ₽

 Россельхозбанк | до 23 %до 21 % | 91 – 1095 дней | от 5000 ₽

 Примсоцбанк | до 22 %до 21.9 % | 212 дней | от 10000 ₽

 Газпромбанк | до 23 % | от 1 дня | 1 – 1500000 ₽

 Интерпрогрессбанк | до 22.01 % | 367 дней | от 100000 ₽

 Альфа-Банк | до 20 % | от 1 дня | 1 – 1500000 ₽

 Газпромбанк | до 19 % | от 1 дня | 1 – 1500000 ₽

 Банк ДОМ.РФ | до 22.7 %до 22.4 % | 91 – 1100 дней | от 30000 ₽

 Кредит Европа Банк (Россия) | до 18.75 % | 91 – 730 дней | от 100000 ₽


### Метод POST

In [6]:
url = 'https://www.sravni.ru/proxy-deposits/products'

data = {
        "productName": "vklady",
        "limit": 300,
        "offset": 0,
        "location": "6.83.",
        "isMix": False,
        "advertisingOnly": False,
        "filters": {
            "group": [
                "organization"
            ],
            "groupLimit": 5,
            "sortProperty": "popularity",
            "sortDirection": "desc",
            "adsAll": True
        }
    }

response = requests.post(url, json=data)
data = response.json()

# Нашел url , который позволяет 

In [7]:
len(data["items"])

189

In [8]:
for item in data["items"]:
    print('\nБанк: ', item["name"])
    print('Ставка: ', item["rate"])
    print('Срок: ', item["term"])
    print(f'Сумма от {item["amount"]["from"]} до {item["amount"]["to"]}')
    print('\nЧисло Дополнительныйх Предложений: ', item["groupCount"])
    if item["groupCount"] > 0:
        for i, group_item in enumerate(item['groupItems'], start=1):  #enumerate - каждому объекту задает индекс
            print(f'\nПредложение {i}')
            print('Ставка: ', group_item["rate"])
            print('Срок: ', group_item["term"])
            print(f'Сумма от {group_item["amount"]["from"]} до {group_item["amount"]["to"]}')


Банк:  ТКБ. Высокий доход_367
Ставка:  22.35 %
Срок:  367 дней
Сумма от 100000 до 3000000

Число Дополнительныйх Предложений:  3

Предложение 1
Ставка:  22.1 %
Срок:  181 день
Сумма от 100000 до 3000000

Предложение 2
Ставка:  21.1 %
Срок:  91 день
Сумма от 100000 до 3000000

Предложение 3
Ставка:  10.25 %
Срок:  1110 дней
Сумма от 100000 до 3000000

Банк:  МКБ. Перспектива (% в конце срока)
Ставка:  до 24 %
Срок:  95 – 1100 дней
Сумма от 10000 до 3000000

Число Дополнительныйх Предложений:  0

Банк:  Альфа-Вклад. Максимальный
Ставка:  до 21.05 %
Срок:  92 – 1095 дней
Сумма от 50000 до None

Число Дополнительныйх Предложений:  3

Предложение 1
Ставка:  до 19.22 %
Срок:  92 – 1095 дней
Сумма от 10000 до None

Предложение 2
Ставка:  до 16.09 %
Срок:  92 – 365 дней
Сумма от 300000 до 200000000

Предложение 3
Ставка:  до 15.99 %
Срок:  92 – 365 дней
Сумма от 300000 до 200000000

Банк:  СмартВклад (повышенная ставка)
Ставка:  до 21 %
Срок:  2 – 24 месяцев
Сумма от 50000 до None

Число Допо

In [9]:
# План:
# 1. Создать две таблицы: 1 таблица - банки, 2 таблица - предложения банка;
# 2. Определить связь междку таблицами;
# 3. Таблицы храним в pandase

In [ ]:
bank_dict = {
    "id": [],
    "name": [],
    "rate": [],
    "online_rate": [],
    "term": [],
    "amount_from": [],
    "amount_to": [],
    "offer_count": []
}

bank_offers_dict = {
    "bank_id": [],
    "bank_name": [],
    "rate": [],
    "online_rate": [],
    "term": [],
    "amount_from": [],
    "amount_to": [],
}


for i, item in enumerate(data["items"], start=1):
    bank_dict["id"].append(i)
    bank_dict["name"].append(item["organization"]["name"]["short"])
    bank_dict["rate"].append(item["rate"])

    # добавляем ставку онлайн банков
    try:
        values = data["items"]["rateTable"]["Онлайн"]["rows"][0]

        percentages = [
            float(percent.replace('*', '').strip('%'))
            for item in values
            for percent in item.split(' / ')
            if '%' in percent
        ]
        bank_dict["online_rate"].append(max(percentages))
    except:
        bank_dict["online_rate"].append(0)

    

    bank_dict["term"].append(item["term"])
    bank_dict["amount_from"].append(item["amount"]["from"])
    bank_dict["amount_to"].append(item["amount"]["to"])
    bank_dict["offer_count"].append(item["groupCount"])


    if item["groupCount"] > 0:
        for group_item in item['groupItems']:  #enumerate - каждому объекту задает индекс
            bank_offers_dict["bank_id"].append(i)
            bank_offers_dict["bank_name"].append(item["organization"]["name"]["short"])
            bank_offers_dict["rate"].append(group_item["rate"])
            bank_offers_dict["term"].append(group_item["term"])
            bank_offers_dict["amount_from"].append(group_item["amount"]["from"])
            bank_offers_dict["amount_to"].append(group_item["amount"]["to"])


### Поиск Наличия Полного Имени Банка

In [11]:
not_existing = [] # хранит несуществующие имен банков

for item in data["items"]:
    # проверяем есть ли короткое имя банка
    try:
        name = item["organization"]["name"]["short"]
    # если в try происходит ошибка -> срабатывает except и добавялем имя банка для которого нет короткого имени (not_existing)
    except:
        not_existing.append(item["name"])

# показывает сколько таких банков
len(not_existing)

0

### Фиксим Ставки для Онлайн Банков
Не все банки имеют онлайн формат. У онлайн банков есть параметр `Онлайн`

In [12]:
# data["items"][0]["rateTable"] # пример не онлайн банка
# data["items"][1]["rateTable"] # пример онлайн банка

In [20]:
row_data = data["items"][2]["rateTable"]["Онлайн"]["rows"][0]

percentages = [
    float(percent.replace('*', '').strip('%'))
    for item in row_data
    for percent in item.split(' / ')
    if '%' in percent
]
max_percentage = max(percentages)
max_percentage



22.0

In [21]:
data["items"][2]["rateTable"]["Онлайн"]["rows"][0]

['от 50 000 ₽',
 '18.71% / 19.01%*',
 '20.47% / 21.01%*',
 '21.05% / 22%*',
 '18.78% / 20.01%*',
 '19.22% / 21%*',
 '17.23% / 19.51%*',
 '16.21% / 18.99%*',
 '16.4% / 20.99%*']

In [36]:
# values = data["items"]["rateTable"]["Онлайн"]["rows"][0]

# percentages = [
#     float(percent.replace('*', '').strip('%'))
#     for item in values
#     for percent in item.split(' / ')
#     if '%' in percent
# ]

# Получаем значения из первой строки первого элемента списка
values = data["items"][0]["rateTable"]["Онлайн"]["rows"][0]  # Это будет строка: ["5% / 6% / 7%"]

# Извлекаем проценты
percentages = [
    float(percent.replace('*', '').strip('%'))
    for item in values  # values - это список, поэтому мы перебираем его
    for percent in item.split(' / ')  # item - это строка, которую мы разбиваем
    if '%' in percent
]

highest_percentage = max(percentages)
print(highest_percentage)

KeyError: 'Онлайн'

In [24]:
df['online_rate'].value_counts()

0    189
Name: online_rate, dtype: int64

In [25]:
df = pd.DataFrame(bank_dict)
df[df['online_rate'] != 0]

,id,name,rate,online_rate,term,amount_from,amount_to,offer_count


In [29]:
pd.DataFrame(bank_dict)

,id,name,rate,online_rate,term,amount_from,amount_to,offer_count
0,1,Транскапиталбанк,22.35 %,0,367 дней,100000,3000000.0,3
1,2,Московский Кредитный Банк,до 24 %,0,95 – 1100 дней,10000,3000000.0,0
2,3,Альфа-Банк,до 21.05 %,0,92 – 1095 дней,50000,NaN,3
3,4,Т-Банк,до 21 %,0,2 – 24 месяцев,50000,NaN,1
4,5,МТС Банк,до 21.57 %,0,91 – 1096 дней,10000,NaN,0
...,...,...,...,...,...,...,...,...
184,185,Спецстройбанк,до 4.5 %,0,30 – 1096 дней,1000000,NaN,0
185,186,СМЛТ Банк,до 4.5 %,0,91 – 367 дней,150000,100000000.0,0
186,187,Чайна Констракшн Банк,до 4 %,0,31 – 90 дней,1000000,NaN,0
187,188,Эко-инвест,до 3 %,0,30 – 365 дней,1,NaN,0


In [28]:
pd.DataFrame(bank_offers_dict)

ValueError: All arrays must be of the same length

In [30]:
df.to_excel('bank_data.xlsx')